# Utiliser l'API Google Books pour récupérer des données de livres

## Partie 1 : Comprendre les API

### a. Qu’est-ce qu’une API ?
- API = Application Programming Interface
- Une API est une interface qui permet à un programme d’accéder aux fonctionnalités ou aux données d’un autre service, sans avoir besoin de connaître son fonctionnement interne.
- Exemple : l’API Google Books met à disposition des données sur des millions de livres, que l’on peut interroger depuis un programme Python.
- On envoie une **requête** (question) à l’API, elle nous répond avec des **données** (souvent en JSON)

### b. Qu’est-ce qu’une requête HTTP ?
- HTTP = HyperText Transfer Protocol – c’est le protocole qui permet à des machines (navigateur, script Python, application mobile...) de communiquer avec des serveurs sur Internet.
- Une requête HTTP est un message envoyé depuis un client (comme un navigateur ou un programme Python) vers un serveur web pour lui demander quelque chose.
- Quand on ouvre un site web ou que l'on demande des données à une API, on envoie une **requête HTTP**, généralement de type :

    - **GET** → pour demander des données
    - **POST** → pour envoyer des données
    - **PUT**, **DELETE**, etc.

- L’API Google Books utilise principalement des requêtes de type GET, comme si on entrait une URL dans un navigateur pour demander des infos.
- Une URL de requête API suit une structure bien définie :

`<URL de base> ? <paramètre1>=<valeur1> & <paramètre2>=<valeur2> & ...`

- URL de base : adresse du point d’entrée de l’API, aussi appelée **endpoint**.
- ? : sépare l’URL de base des paramètres de requête.
- Paramètres de requête (query parameters) : ajoutés sous forme de paires clé=valeur. Chaque paramètre est séparé par &.  Les paramètres possibles sont disponibles dans la documentation de chaque API.

### c. Réponse

Quand on envoie une requête GET à une API, on demande au serveur de renvoyer une ressource.

La réponse de ce serveur comporte deux parties essentielles :


1. **Le code de status de réponse (HTTP status code)**

C’est un nombre à 3 chiffres qui indique si la requête a réussi ou non.

--> [Liste des codes possibles](https://developer.mozilla.org/fr/docs/Web/HTTP/Reference/Status) et leur signification


2. **Le contenu de la réponse (le "corps")**

C’est ici que se trouvent les données demandées.

Ce contenu est souvent au format JSON (JavaScript Object Notation), c’est-à-dire une structure clé-valeur que Python peut facilement lire.

Exemple d'un format JSON :
```json
{
  "items": [
    {
      "tripInfo": {
        "country": "France",
        "cities": ["Montpellier", "Marseille"],
        "nb_photo_taken": 200,
        "language": "fr"
      }
    }
  ]
}
```

---
## Partie 2 : Requêter Google Books API

Dans cette partie vous allez : 
- Faire une requête HTTP pour interroger l'API Google Books à l'aide de la bibliothèque python _requests_,
- Vérifier le code de status de la réponse,
- Traiter le corps de la réponse à l'aide de la bibliothèque python _json_,
- Utiliser _pandas_ pour convertir les données reçues en dataframe.


In [89]:
# Importer les bibliothèques nécessaires
import requests
import json
import pandas as pd

Pour interroger une API, un certain nombre de paramètres et filtres définis dans la documentation peuvent être utilisés. 

En utilisant la [documentation](https://developers.google.com/books/docs/v1/using?hl=fr#WorkingVolumes) de l'API Google Books, définir le dictionnaire de paramètres permettant de faire une recherche sur :
- la chaîne de texte : "food",
- il doit s'agir d'un e-book avec un prix d'achat,
- le nombre maximal de résultats à renvoyer est de 40,
- les résultats doivent être triés par pertinence.

In [90]:
# URL de l'API Google Books
url = "https://www.googleapis.com/books/v1/volumes"

# Dictionnaire de paramètres pour la requête 
params = {"q": "food",
          "filter": "paid-ebooks",
          "maxResults": 40,
          "orderBy": "relevance"
          }

#url = "https://www.googleapis.com/books/v1/volumes?q=food&filter=paid-ebooks&maxResults=40&orderBy:relevance"

En utilisant la documentation de la bibliothèque [_requests_](https://requests.readthedocs.io/en/latest/user/quickstart/):
- implémenter la requête HTTP _get_  avec les paramètres sur l'URL définie ci-dessus,
- vérifier le code de réponse HTTP.

In [91]:
# Requêter l'API Google Books
response = requests.get(url, params=params)

# Vérifier le code de statut de la réponse
print(response.status_code)

#Ici on envoie une requête HTTP GET vers l’API Google Books
#La bibliothèque requests va construire automatiquement l’URL complète : "https://www.googleapis.com/books/v1/volumes?q=food&filter=paid-ebooks&maxResults=40&orderBy:relevance"
#Puis elle envoie la requête au serveur Google.
#Le résultat est stocké dans l’objet response.


200


Récupérer et observer le coeur de la réponse via l'objet _response_ créé précédemment.

In [92]:
# Récupérer le coeur de la réponse
data_books_raw = response.json()

# Afficher les données récupérées
#print(data_books_raw)

# Afficher le type de la variable data_books_raw
#print(type(data_books_raw))

#La méthode .json() de l’objet response :
#lit le corps de la réponse HTTP (qui est en format JSON, c’est-à-dire du texte structuré)
#et le convertit en dictionnaire Python (dict) automatiquement.

{
  "kind": "books#volumes",
  "totalItems": 123,
  "items": [ {...} ]
}

#print(data_books_raw.keys())
print(json.dumps(data_books_raw, indent=2))

#Donc en résumé :
#response.json() → convertit le JSON en dict Python
#print(...) → affiche les données
#type(...) → confirme la structure (dict)

{
  "kind": "books#volumes",
  "totalItems": 1000000,
  "items": [
    {
      "kind": "books#volume",
      "id": "O-t9BAAAQBAJ",
      "etag": "D9+ivUDCftU",
      "selfLink": "https://www.googleapis.com/books/v1/volumes/O-t9BAAAQBAJ",
      "volumeInfo": {
        "title": "Encyclopedia of Food and Health",
        "publisher": "Academic Press",
        "publishedDate": "2015-08-26",
        "description": "Approx.3876 pages Approx.3876 pages",
        "industryIdentifiers": [
          {
            "type": "ISBN_13",
            "identifier": "9780123849533"
          },
          {
            "type": "ISBN_10",
            "identifier": "0123849535"
          }
        ],
        "readingModes": {
          "text": false,
          "image": true
        },
        "pageCount": 2379,
        "printType": "BOOK",
        "categories": [
          "Technology & Engineering"
        ],
        "maturityRating": "NOT_MATURE",
        "allowAnonLogging": false,
        "contentVersion

- Les données relatives aux livres récupérées sont accessibles dans la clé 'items',
- Récupérer les données relatives aux livres dans la variable data_books.

**NOTE**

Il existe 2 méthodes pour accéder aux valeurs d'une clé ("nom_de_la_clé") d'un dictionnaire python : 
- `mon_dict["nom_de_la_clé"]`: si "nom_de_la_clé" n'est pas présent dans le dictionnaire, cela retourne une erreur.
- `mon_dict.get("nom_de_la_clé")` : si "nom_de_la_clé" n'est pas présent dans le dictionnaire, cela renvoie None par défault. 
Il est aussi possible d'indiquer comme second argument la variable à renvoyer si la clé n'est pas présente : 

`mon_dict.get("nom_de_la_clé", "clé inexistante")`

Privilégier la seconde méthode car certaines clés peuvent ne pas exister.

--> [Lien ressource](https://w3schools.tech/fr/tutorial/python/python_access_dictionary_items) : accès au clé/valeurs d'un dictionnaire python

In [93]:
# Récupérer les données relatives aux livres via la clé 'items'
data_books = data_books_raw["items"]

# Afficher le type de la variable data_books
print(type(data_books))

print(json.dumps(data_books, indent=2))

<class 'list'>
[
  {
    "kind": "books#volume",
    "id": "O-t9BAAAQBAJ",
    "etag": "D9+ivUDCftU",
    "selfLink": "https://www.googleapis.com/books/v1/volumes/O-t9BAAAQBAJ",
    "volumeInfo": {
      "title": "Encyclopedia of Food and Health",
      "publisher": "Academic Press",
      "publishedDate": "2015-08-26",
      "description": "Approx.3876 pages Approx.3876 pages",
      "industryIdentifiers": [
        {
          "type": "ISBN_13",
          "identifier": "9780123849533"
        },
        {
          "type": "ISBN_10",
          "identifier": "0123849535"
        }
      ],
      "readingModes": {
        "text": false,
        "image": true
      },
      "pageCount": 2379,
      "printType": "BOOK",
      "categories": [
        "Technology & Engineering"
      ],
      "maturityRating": "NOT_MATURE",
      "allowAnonLogging": false,
      "contentVersion": "2.6.5.0.preview.1",
      "panelizationSummary": {
        "containsEpubBubbles": false,
        "containsImag

Accéder au premier élément de _data_books_ pour afficher les données du premier livre.

--> [Lien ressource](https://w3schools.tech/fr/tutorial/python/python_access_list_items) : accéder aux éléments d'une liste python

In [94]:
# Afficher le premier livre
print(json.dumps(data_books[0], indent=2))

{
  "kind": "books#volume",
  "id": "O-t9BAAAQBAJ",
  "etag": "D9+ivUDCftU",
  "selfLink": "https://www.googleapis.com/books/v1/volumes/O-t9BAAAQBAJ",
  "volumeInfo": {
    "title": "Encyclopedia of Food and Health",
    "publisher": "Academic Press",
    "publishedDate": "2015-08-26",
    "description": "Approx.3876 pages Approx.3876 pages",
    "industryIdentifiers": [
      {
        "type": "ISBN_13",
        "identifier": "9780123849533"
      },
      {
        "type": "ISBN_10",
        "identifier": "0123849535"
      }
    ],
    "readingModes": {
      "text": false,
      "image": true
    },
    "pageCount": 2379,
    "printType": "BOOK",
    "categories": [
      "Technology & Engineering"
    ],
    "maturityRating": "NOT_MATURE",
    "allowAnonLogging": false,
    "contentVersion": "2.6.5.0.preview.1",
    "panelizationSummary": {
      "containsEpubBubbles": false,
      "containsImageBubbles": false
    },
    "imageLinks": {
      "smallThumbnail": "http://books.googl

In [95]:
with open("show_books.json", "w", encoding="utf-8") as f:
    json.dump(data_books_raw, f, indent=2, ensure_ascii=False)

Dans les données du premier livre, chercher où se trouve les informations suivantes (il peut s'agir de dictionnaires imbriqués) :
- le titre,
- le prix,
- la note (_averageRating_)

Afficher ces informations (cf la note ci-dessus pour accéder aux éléments d'un dictionnaire en python).

In [96]:
title_first_book = data_books[0]["volumeInfo"]["title"]
price_first_book = data_books[0]["saleInfo"]["listPrice"]["amount"]
average_rating_first_book = data_books[0]["volumeInfo"].get("averageRating", "nonexistante")
print(f"Titre : {title_first_book}")
print(f"Prix : {price_first_book}")
print(f"Note : {average_rating_first_book}")

Titre : Encyclopedia of Food and Health
Prix : 3334
Note : nonexistante


In [97]:
# Afficher les infos clés (price, title, rating) pour chaque livres de la liste data_books
for book in data_books:
        volume_info = book.get("volumeInfo", {})
        sale_info = book.get("saleInfo", {})

        title = volume_info.get("title")
        price = sale_info.get("listPrice", {}).get("amount")
        rating = volume_info.get("averegeRating")

        print(f"Titre : {title}")
        print(f"Prix : {price}")
        print(f"Note : {rating}")
        

#The {} here is just the default value in case the key is missing.
#book.get("volumeInfo", {}) → looks for the key "volumeInfo".
#If it exists → returns its dictionary.
#If it does not exist → returns {}

Titre : Encyclopedia of Food and Health
Prix : 3334
Note : None
Titre : When Technology Fails
Prix : 27.82
Note : None
Titre : Advances in Deep-Fat Frying of Foods
Prix : 274.3
Note : None
Titre : Solid State Fermentation for Foods and Beverages
Prix : 90.72
Note : None
Titre : In-Pack Processed Foods
Prix : 226.82
Note : None
Titre : Current Advances for Development of Functional Foods Modulating Inflammation and Oxidative Stress
Prix : 184.62
Note : None
Titre : Fermented Foods and Beverages of the World
Prix : 211
Note : None
Titre : Biopolymers in Nutraceuticals and Functional Foods
Prix : 236.32
Note : None
Titre : Fermentation Biotechnology for Functional Foods
Prix : 247.92
Note : None
Titre : Functional Foods and their Implications for Health Promotion
Prix : 163.52
Note : None
Titre : Strange Foods
Prix : 10.99
Note : None
Titre : Olives and Olive Oil as Functional Foods
Prix : 174.99
Note : None
Titre : Issues of Governance, Security, and Development in Contemporary Africa
Pr

Créer une liste de dictionnaire contenant les informations de chaque livre sous le format : 
```
[{
    "title": "title0",
    "price": price_0,
    "rating": rating_0
},
{
    "title": "title1",
    "price": price_1,
    "rating": rating_1
},
...
]
```

In [98]:
# Création d'une liste de dictionnaires pour les livres
books_list = []

for book in data_books:
        volume_info = book.get("volumeInfo", {})
        sale_info = book.get("saleInfo", {})

        title = volume_info.get("title")
        price = sale_info.get("listPrice", {}).get("amount")
        rating = volume_info.get("averegeRating")

        book_dict = {
                "title" : title,
                "price" : price,
                "rating" : rating
        }

        books_list.append(book_dict)
       

In [99]:
# Créer un dataframe à partir de la liste de dictionnaires
df = pd.DataFrame(books_list)
print(df.head())

                                              title    price rating
0                   Encyclopedia of Food and Health  3334.00   None
1                             When Technology Fails    27.82   None
2              Advances in Deep-Fat Frying of Foods   274.30   None
3  Solid State Fermentation for Foods and Beverages    90.72   None
4                           In-Pack Processed Foods   226.82   None


Filtrer les livres pour ne conserver que ceux ayant des valeurs pour les colonnes _price_ et _rating_.

- Possibilité d'utiliser la méthode pandas [_dropna_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)
- Afficher le dataframe et regarder ce qu'il s'est passé au niveau des index.
- Utiliser la méthode [_reset_index_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html) pour réinitialiser les index.

In [100]:
# Filter les livres pour ne conserver que ceux ayant des valeurs pour les colonnes price et rating
df = df.copy()
df_clean = df.dropna(subset=["price", "rating"])

# Reinitialiser les index du DataFrame
df_clean = df_clean.reset_index(drop=True)
print(df_clean.head())

# Ajouter une colonne availability = False
df_clean["availability"] = False


Empty DataFrame
Columns: [title, price, rating]
Index: []


---
## Partie 3 : Insertion des données en base

In [103]:
# Utiliser sqlite3 pour insérer les données dans la base de données book_store dans la table book
import sqlite3

connection = sqlite3.connect("/home/emese/Briefs_test/scraping/book_store.db")
df_clean.to_sql('book_store', connection, if_exists='replace')
connection.commit()

In [104]:
# Vérifier que le nombre de livres dans la table correspond au nb de livres scrapés + livres de l'API
# Création d'un curseur pour interagir avec la DB
cursor = connection.cursor()

# Exécuter la requête pour compter le nombre de livre dans la DB
cursor.execute("SELECT COUNT(*) FROM book_store")

# Affichage du résultat
print(cursor.fetchone()[0])


connection.close()

0
